In [ ]:
# this cell is only for colab execution
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
# this cell is only for colab execution
%cd ../content/drive/MyDrive/SymbolicMathematics-master/

/content/drive/MyDrive/SymbolicMathematics-master


In [ ]:
ls

beam_integration.ipynb  CONTRIBUTING.md  LICENSE  ode1.pth  README.md      src/
CODE_OF_CONDUCT.md      fwd_bwd.pth      main.py  ode2.pth  split_data.py


In [ ]:
import os
import numpy as np
import sympy as sp
import torch

from src.utils import AttrDict
from src.envs import build_env
from src.model import build_modules

from src.utils import to_cuda
from src.envs.sympy_utils import simplify

## Build environment / Reload model

In [ ]:
# trained model
#!wget https://dl.fbaipublicfiles.com/SymbolicMathematics/models/ode1.pth
model_path = 'ode1.pth'
assert os.path.isfile(model_path)

In [ ]:
params = params = AttrDict({

    # environment parameters
    'env_name': 'char_sp',
    'int_base': 10,
    'balanced': False,
    'positive': True,
    'precision': 10,
    'n_variables': 1,
    'n_coefficients': 1,
    'leaf_probs': '0.6,0.2,0.2,0',
    'max_len': 512,
    'max_int': 5,
    'max_ops': 15,
    'max_ops_G': 15,
    'clean_prefix_expr': True,
    'rewrite_functions': '',
    'tasks': 'ode1',
    'operators': 'add:10,sub:3,mul:10,div:5,sqrt:4,pow2:4,pow3:2,pow4:1,pow5:1,ln:4,exp:4,sin:4,cos:4,tan:4,asin:1,acos:1,atan:1,sinh:1,cosh:1,tanh:1,asinh:1,acosh:1,atanh:1',

    # model parameters
    'cpu': False,
    'emb_dim': 1024,
    'n_enc_layers': 6,
    'n_dec_layers': 6,
    'n_heads': 8,
    'dropout': 0,
    'attention_dropout': 0,
    'sinusoidal_embeddings': False,
    'share_inout_emb': True,
    'reload_model': './ode1.pth',    

})

In [ ]:
# building the enviroment
env = build_env(params)
x = env.local_dict['x']
f = env.local_dict['f']
a0 = env.local_dict['a0']
a8 = env.local_dict['a8']

modules = build_modules(env, params)
encoder = modules['encoder']
decoder = modules['decoder']

In [ ]:
# this is an easy ODE problem just for testing
ode = sp.Derivative(f(x))

display(ode)

Derivative(f(x), x)

In [ ]:
ode_prefix = env.sympy_to_prefix(ode)
x1_prefix = env.clean_prefix(ode_prefix)

x1 = torch.LongTensor(
    [env.eos_index] +
    [env.word2id[w] for w in x1_prefix] +
    [env.eos_index]
).view(-1, 1)
len1 = torch.LongTensor([len(x1)])
x1, len1 = to_cuda(x1, len1)

with torch.no_grad():
    encoded = encoder('fwd', x=x1, lengths=len1, causal=False).transpose(0, 1)

In [ ]:
beam_size = 10
with torch.no_grad():
    _, _, beam = decoder.generate_beam(encoded, len1, beam_size=beam_size, length_penalty=1.0, early_stopping=1, max_len=200)
    assert len(beam) == 1
hypotheses = beam[0].hyp
assert len(hypotheses) == beam_size

In [ ]:
print("Input function ODE:\n")
display(ode)
print("")

firstCorrect = None
for score, sent in sorted(hypotheses, key=lambda x: x[0], reverse=True):

    # parse decoded hypothesis
    ids = sent[1:].tolist()                  # decoded token IDs
    tok = [env.id2word[wid] for wid in ids]  # convert to prefix

    try:
        hyp = env.prefix_to_infix(tok)       # convert to infix
        hyp = env.infix_to_sympy(hyp)        # convert to SymPy

        # check whether we recover f if we differentiate the hypothesis
        res = "OK" if simplify(simplify(ode.subs(f(x), hyp), seconds=1), seconds=1) == 0 else "NO"
        if (firstCorrect==None and res=="OK"):
            firstCorrect = hyp

    except:
        res = "INVALID PREFIX EXPRESSION"
        hyp = tok

    # print result
    print("%.5f  %s  %s" % (score, res, hyp))
        
print("\nThe well-predicted expression with the highest score is:\n")
display(firstCorrect)

Input function ODE:



Derivative(f(x), x)


-0.41214  OK  (a8*x + x/2)/(2*x)
-0.41499  OK  (a8*x + x/4)/(2*x)
-0.42488  OK  (a8*x + x/3)/(2*x)
-0.49265  OK  (a8*x + 2*x)*log(4)/(2*x)
-0.49470  OK  (a8*x + 2*x)*sin(1)/(2*x)
-0.49643  OK  (a8*x + 2*x)*tan(1)/(2*x)
-0.49689  OK  (a8*x + x)*log(4)/(2*x)
-0.49692  OK  (a8*x + 2*x)*log(2)/(2*x)
-0.49817  OK  (a8*x + x)*tan(1)/(2*x)
-0.49829  OK  (a8*x + x)*sin(1)/(2*x)

The well-predicted expression with the highest score is:



(a8*x + x/2)/(2*x)

In [ ]:
from sympy.solvers import solve

def sol_with_initial_condition(sol, i_c):
  sol=sp.simplify(sol)
  I = sol.subs(x,0)
  i = solve(I-i_c,a8)[0]
  Phi  = sol.subs(a8,i)
  return sp.simplify(Phi)

In [ ]:
display(sol_with_initial_condition(firstCorrect, 1))

cos(x)